# Program Pengenalan Tanda Lalu Lintas
## Kelompok
#### Giast Ahmad
#### Alif Al Husaini
#### Angga Prasetyo
#### M Danendra
#### RIo Irawan


## Import Libraries

In [25]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
from sklearn.model_selection import train_test_split
import pickle
import os
import shutil
import pandas as pd
import random
import tensorflow as tf

In [24]:
def create_dir_if_not_exists(directory):
    if not os.path.exists(directory):
        os.makedirs(directory)

In [26]:
def split_data(source_dir, train_dir, test_dir, test_size=0.2):
    labels = os.listdir(source_dir)
    
    for label in labels:
        label_dir = os.path.join(source_dir, label)
        if not os.path.isdir(label_dir):
            continue
        
        images = os.listdir(label_dir)
        random.shuffle(images)
        
        num_images = len(images)
        num_test = int(test_size * num_images)
        num_train = num_images - num_test
        
        train_images = images[:num_train]
        test_images = images[num_train:]
        
        train_label_dir = os.path.join(train_dir, label)
        test_label_dir = os.path.join(test_dir, label)
        
        create_dir_if_not_exists(train_label_dir)
        create_dir_if_not_exists(test_label_dir)
        
        for image in train_images:
            shutil.copy(os.path.join(label_dir, image), os.path.join(train_label_dir, image))
        
        for image in test_images:
            shutil.copy(os.path.join(label_dir, image), os.path.join(test_label_dir, image))
            
        print(f"Label {label}: {num_train} images for training, {num_test} images for testing")

In [27]:
# Define directories
source_directory = 'myData'
train_directory = 'myDataTrain'
test_directory = 'myDataTest'

# Create train and test directories if they do not exist
create_dir_if_not_exists(train_directory)
create_dir_if_not_exists(test_directory)

# Split data
split_data(source_directory, train_directory, test_directory, test_size=0.2)

Label 0: 144 images for training, 36 images for testing
Label 1: 1584 images for training, 396 images for testing
Label 10: 1440 images for training, 360 images for testing
Label 11: 936 images for training, 234 images for testing
Label 12: 1512 images for training, 378 images for testing
Label 13: 1536 images for training, 384 images for testing
Label 14: 552 images for training, 138 images for testing
Label 15: 432 images for training, 108 images for testing
Label 16: 288 images for training, 72 images for testing
Label 17: 792 images for training, 198 images for testing
Label 18: 864 images for training, 216 images for testing
Label 19: 144 images for training, 36 images for testing
Label 2: 1608 images for training, 402 images for testing
Label 20: 240 images for training, 60 images for testing
Label 21: 216 images for training, 54 images for testing
Label 22: 264 images for training, 66 images for testing
Label 23: 360 images for training, 90 images for testing
Label 24: 192 image

In [28]:
import os
import cv2
import csv

def create_dir_if_not_exists(directory):
    if not os.path.exists(directory):
        os.makedirs(directory)

def find_roi(image):
    # Convert image to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Apply Gaussian blur to reduce noise
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)
    
    # Detect edges using Canny
    edges = cv2.Canny(blurred, 50, 150)
    
    # Find contours
    contours, _ = cv2.findContours(edges.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    if len(contours) > 0:
        # Find the largest contour
        max_contour = max(contours, key=cv2.contourArea)
        
        # Get bounding box of the largest contour
        x, y, w, h = cv2.boundingRect(max_contour)
        
        return x, y, x + w, y + h
    
    return None

def create_csv(file_path, image_dir):
    # Define the CSV header
    header = ['Width', 'Height', 'Roi.X1', 'Roi.Y1', 'Roi.X2', 'Roi.Y2', 'ClassId', 'Path']
    
    # Open the CSV file for writing
    with open(file_path, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(header)
        
        # Iterate through each label directory
        for label in os.listdir(image_dir):
            label_dir = os.path.join(image_dir, label)
            if not os.path.isdir(label_dir):
                continue
            
            # Iterate through each image in the label directory
            for image_name in os.listdir(label_dir):
                image_path = os.path.join(label_dir, image_name)
                if not os.path.isfile(image_path):
                    continue
                
                # Read the image
                image = cv2.imread(image_path)
                if image is None:
                    continue
                
                height, width = image.shape[:2]
                
                # Find ROI
                roi = find_roi(image)
                if roi is not None:
                    x1, y1, x2, y2 = roi
                else:
                    # If ROI is not found, use the entire image
                    x1, y1, x2, y2 = 0, 0, width, height
                
                # Write the row to the CSV file
                writer.writerow([width, height, x1, y1, x2, y2, label, image_path])
                
    print(f'CSV file created at: {file_path}')

# Define directories
train_directory = 'myDataTrain'
test_directory = 'myDataTest'
train_csv = 'train.csv'
test_csv = 'test.csv'

# Create CSV files for train and test sets
create_csv(train_csv, train_directory)
create_csv(test_csv, test_directory)


CSV file created at: train.csv
CSV file created at: test.csv


In [30]:
from keras.utils import to_categorical
def load_data(data_dir):
    images = []
    labels = []
    
    # Iterate through each label directory
    for label in os.listdir(data_dir):
        label_dir = os.path.join(data_dir, label)
        
        # Iterate through each image in the label directory
        for image_name in os.listdir(label_dir):
            image_path = os.path.join(label_dir, image_name)
            image = cv2.imread(image_path)
            if image is not None:
                images.append(image)
                labels.append(int(label))
    
    return np.array(images), np.array(labels)

# Directory paths for train and test sets
train_directory = 'myDataTrain'
test_directory = 'myDataTest'

# Load train and test data
X_train, y_train = load_data(train_directory)
X_test, y_test = load_data(test_directory)

# Normalize pixel values to range [0, 1]
X_train = X_train.astype('float32') / 255
X_test = X_test.astype('float32') / 255

# One-hot encode the labels
num_classes = len(np.unique(y_train))
y_train = to_categorical(y_train, num_classes)
y_test = to_categorical(y_test, num_classes)


In [31]:
import os
import numpy as np
import cv2
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras.optimizers import Adam
from keras.utils import to_categorical

model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=X_train.shape[1:]),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(num_classes, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
history = model.fit(X_train, y_train, batch_size=32, epochs=10, validation_data=(X_test, y_test))


c:\Users\Alif\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
870/870 ━━━━━━━━━━━━━━━━━━━━ 23s 24ms/step - accuracy: 0.4334 - loss: 2.0901 - val_accuracy: 0.9316 - val_loss: 0.2452
Epoch 2/10
870/870 ━━━━━━━━━━━━━━━━━━━━ 21s 24ms/step - accuracy: 0.9163 - loss: 0.2854 - val_accuracy: 0.9793 - val_loss: 0.0958
Epoch 3/10
870/870 ━━━━━━━━━━━━━━━━━━━━ 23s 26ms/step - accuracy: 0.9557 - loss: 0.1505 - val_accuracy: 0.9836 - val_loss: 0.0673
Epoch 4/10
870/870 ━━━━━━━━━━━━━━━━━━━━ 22s 25ms/step - accuracy: 0.9731 - loss: 0.0943 - val_accuracy: 0.9839 - val_loss: 0.0696
Epoch 5/10
870/870 ━━━━━━━━━━━━━━━━━━━━ 21s 24ms/step - accuracy: 0.9785 - loss: 0.0701 - val_accuracy: 0.9862 - val_loss: 0.0567
Epoch 6/10
870/870 ━━━━━━━━━━━━━━━━━━━━ 23s 27ms/step - accuracy: 0.9808 - loss: 0.0615 - val_accuracy: 0.9886 - val_loss: 0.0455
Epoch 7/10
870/870 ━━━━━━━━━━━━━━━━━━━━ 21s 24ms/step - accuracy: 0.9876 - loss: 0.0426 - val_accuracy: 0.9941 - val_loss: 0.0320
Epoch 8/10
870/870 ━━━━━━━━━━━━━━━━━━━━ 21s 24ms/step - accuracy: 0.9903 - loss: 0.0318 - 

In [32]:
model.save('your_model.h5') 

In [33]:
import os
import cv2
import numpy as np
from keras.models import load_model

# Load trained model
model = load_model('your_model.h5')  # Ganti 'your_trained_model.h5' dengan nama file model Anda

# Function to preprocess image
def preprocess_image(image_path):
    image = cv2.imread(image_path)
    image = cv2.resize(image, (32, 32))  # Resize image to match input shape of the model
    image = image.astype('float32') / 255  # Normalize pixel values
    image = np.expand_dims(image, axis=0)  # Add batch dimension
    return image

# Directory path for test set
test_directory = 'myDataTest'

# Iterate through each label directory
for label in os.listdir(test_directory):
    label_dir = os.path.join(test_directory, label)
    
    # Iterate through each image in the label directory
    for image_name in os.listdir(label_dir):
        image_path = os.path.join(label_dir, image_name)
        
        # Preprocess image
        image = preprocess_image(image_path)
        
        # Predict class probabilities
        predictions = model.predict(image)
        
        # Get predicted class index
        predicted_class = np.argmax(predictions)
        
        # Print predicted class and corresponding probability
        print(f'Image: {image_name}, Predicted Class: {predicted_class}, Probability: {predictions[0][predicted_class]}')


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
Image: 0_10006_1577671998.6541514.png, Predicted Class: 0, Probability: 0.9999943971633911
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
Image: 0_10018_1577671998.6621292.png, Predicted Class: 0, Probability: 0.9999486207962036
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
Image: 0_10023_1577671998.66512.png, Predicted Class: 0, Probability: 0.6208482384681702
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
Image: 0_10026_1577671998.6671147.png, Predicted Class: 0, Probability: 0.9998608827590942
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
Image: 0_10028_1577671998.6691096.png, Predicted Class: 0, Probability: 0.9996376037597656
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
Image: 0_10030_1577671998.67011.png, Predicted Class: 0, Probability: 0.9999932050704956
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
Image: 0_10031_1577671998.6711066.png, Predicted Class: 0, Probability: 0.9999979734420776
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
Image: 0_10042_1577671998.6790838.png, Predicted Class: 0, Prob

KeyboardInterrupt: 

In [34]:
import cv2
import numpy as np
from keras.models import load_model

# Load trained model
model = load_model('your_model.h5')  # Ganti 'your_trained_model.h5' dengan nama file model Anda

# Function to preprocess image
def preprocess_image(image):
    image = cv2.resize(image, (32, 32))  # Resize image to match input shape of the model
    image = image.astype('float32') / 255  # Normalize pixel values
    image = np.expand_dims(image, axis=0)  # Add batch dimension
    return image

# Function to classify image using the model
def classify_image(image):
    preprocessed_image = preprocess_image(image)
    predictions = model.predict(preprocessed_image)
    predicted_class = np.argmax(predictions)
    return predicted_class, predictions[0][predicted_class]

# Function to process video stream from camera
def process_video_stream():
    # Open video capture device (camera)
    cap = cv2.VideoCapture(0)

    while True:
        # Read frame from the camera
        ret, frame = cap.read()
        if not ret:
            break

        # Classify frame
        predicted_class, confidence = classify_image(frame)

        # Display class label and confidence on the frame
        label = f'Class: {predicted_class}, Confidence: {confidence:.2f}'
        cv2.putText(frame, label, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

        # Display the frame
        cv2.imshow('Camera', frame)

        # Exit loop if 'q' is pressed
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # Release the camera and close OpenCV windows
    cap.release()
    cv2.destroyAllWindows()

# Call the function to process video stream
process_video_stream()


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 94ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 109ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━

KeyboardInterrupt: 

In [1]:
import os
import cv2

# Fungsi untuk mendeteksi objek menggunakan detektor tepi Canny
def detect_objects(image):
    # Ubah gambar menjadi grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Deteksi tepi menggunakan detektor Canny
    edges = cv2.Canny(gray, 50, 150)

    # Temukan kontur objek dari tepi yang terdeteksi
    contours, _ = cv2.findContours(edges.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Filter kontur berdasarkan area
    min_area = 100  # Ubah sesuai dengan kebutuhan
    max_area = 10000  # Ubah sesuai dengan kebutuhan
    detected_objects = []
    for contour in contours:
        area = cv2.contourArea(contour)
        if min_area < area < max_area:
            x, y, w, h = cv2.boundingRect(contour)
            detected_objects.append((x, y, x + w, y + h))  # Format: (x1, y1, x2, y2)

    return detected_objects

# Fungsi untuk menampilkan gambar dan melakukan anotasi objek secara otomatis
def annotate_image(image_path, detected_objects):
    # Baca gambar
    image = cv2.imread(image_path)
    if image is None:
        print(f"Failed to read image: {image_path}")
        return

    # Tampilkan gambar
    cv2.imshow("Image", image)

    # Tampilkan anotasi objek pada gambar
    for obj in detected_objects:
        x1, y1, x2, y2 = obj
        cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)

    # Tampilkan gambar dengan anotasi objek
    cv2.imshow("Annotated Image", image)

    # Tunggu tombol 'q' ditekan untuk keluar
    while True:
        key = cv2.waitKey(0)
        if key == ord('q'):
            break

    cv2.destroyAllWindows()

# Path ke direktori dataset
dataset_dir = "myData"

# Loop melalui setiap folder (label)
for label in os.listdir(dataset_dir):
    label_dir = os.path.join(dataset_dir, label)
    if not os.path.isdir(label_dir):
        continue

    # Loop melalui setiap gambar dalam folder
    for image_name in os.listdir(label_dir):
        image_path = os.path.join(label_dir, image_name)

        # Baca gambar
        image = cv2.imread(image_path)

        # Deteksi objek menggunakan detektor tepi Canny
        detected_objects = detect_objects(image)

        # Annotasi gambar dengan objek yang terdeteksi
        annotate_image(image_path, detected_objects)


KeyboardInterrupt: 

In [6]:
import cv2
import numpy as np
import os

# Load model YOLO dari disk
net = cv2.dnn.readNet("yolov3.weights", "yolov3.cfg")
layer_names = net.getLayerNames()
output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers()]

# Path ke direktori dataset
dataset_dir = "myData"

# Path ke direktori untuk menyimpan anotasi
annotation_dir = "annotations"

# Loop melalui setiap folder (label)
for label in os.listdir(dataset_dir):
    label_dir = os.path.join(dataset_dir, label)
    if not os.path.isdir(label_dir):
        continue

    # Loop melalui setiap gambar dalam folder
    for image_name in os.listdir(label_dir):
        image_path = os.path.join(label_dir, image_name)

        # Baca gambar
        img = cv2.imread(image_path)
        height, width, channels = img.shape

        # Konversi gambar menjadi blob
        blob = cv2.dnn.blobFromImage(img, 0.00392, (416, 416), (0, 0, 0), True, crop=False)

        # Masukkan blob ke dalam model YOLO
        net.setInput(blob)
        outs = net.forward(output_layers)

        # Proses output dari YOLO
        for out in outs:
            for detection in out:
                scores = detection[5:]
                class_id = np.argmax(scores)
                confidence = scores[class_id]
                if confidence > 0.5 and class_id == 9:  # ID kelas untuk traffic sign adalah 9
                    # Hitung koordinat kotak pembatas
                    center_x = int(detection[0] * width)
                    center_y = int(detection[1] * height)
                    w = int(detection[2] * width)
                    h = int(detection[3] * height)
                    x = int(center_x - w / 2)
                    y = int(center_y - h / 2)

                    # Simpan anotasi ke file
                    annotation_file = os.path.join(annotation_dir, f"{label}_{image_name}.txt")
                    with open(annotation_file, 'w') as f:
                        f.write(f"{x},{y},{x+w},{y+h}\n")


KeyboardInterrupt: 

In [7]:
import numpy as np
import cv2
import pickle
from keras.models import load_model

# SETUP THE VIDEO CAMERA
frameWidth = 640
frameHeight = 480
brightness = 180
cap = cv2.VideoCapture(0)
cap.set(3, frameWidth)
cap.set(4, frameHeight)
cap.set(10, brightness)

# IMPORT THE TRAINED MODEL
model = load_model("traffic_sign_model.h5")

def grayscale(img):
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    return img

def equalize(img):
    img = cv2.equalizeHist(img)
    return img

def preprocessing(img):
    img = grayscale(img)
    img = equalize(img)
    img = img / 255
    return img

def getClassName(classNo):
    classNames = {
        0: 'Speed Limit 20 km/h',
        1: 'Speed Limit 30 km/h',
        2: 'Speed Limit 50 km/h',
        3: 'Speed Limit 60 km/h',
        4: 'Speed Limit 70 km/h',
        5: 'Speed Limit 80 km/h',
        6: 'End of Speed Limit 80 km/h',
        7: 'Speed Limit 100 km/h',
        8: 'Speed Limit 120 km/h',
        9: 'No passing',
        10: 'No passing for vechiles over 3.5 metric tons',
        11: 'Right-of-way at the next intersection',
        12: 'Priority road',
        13: 'Yield',
        14: 'Stop',
        15: 'No vehicles',
        16: 'Vehicles over 3.5 metric tons prohibited',
        17: 'No entry',
        18: 'General caution',
        19: 'Dangerous curve to the left',
        20: 'Dangerous curve to the right',
        21: 'Double curve',
        22: 'Bumpy road',
        23: 'Slippery road',
        24: 'Road narrows on the right',
        25: 'Road work',
        26: 'Traffic signals',
        27: 'Pedestrians',
        28: 'Children crossing',
        29: 'Bicycles crossing',
        30: 'Beware of ice/snow',
        31: 'Wild animals crossing',
        32: 'End of all speed and passing limits',
        33: 'Turn right ahead',
        34: 'Turn left ahead',
        35: 'Ahead only',
        36: 'Go straight or right',
        37: 'Go straight or left',
        38: 'Keep right',
        39: 'Keep left',
        40: 'Roundabout mandatory',
        41: 'End of no passing',
        42: 'End of no passing by vechiles over 3.5 metric tons'
    }
    return classNames[classNo]

font = cv2.FONT_HERSHEY_SIMPLEX
threshold = 0.75  # You might want to adjust this threshold

while True:
    # READ IMAGE
    success, imgOrignal = cap.read()
    
    # PROCESS IMAGE
    img = np.asarray(imgOrignal)
    img = cv2.resize(img, (32, 32))
    img = preprocessing(img)
    cv2.imshow("Processed Image", img)
    img = img.reshape(1, 32, 32, 1)
    cv2.putText(imgOrignal, "CLASS: " , (20, 35), font, 0.75, (0, 0, 255), 2, cv2.LINE_AA)
    cv2.putText(imgOrignal, "PROBABILITY: ", (20, 75), font, 0.75, (0, 0, 255), 2, cv2.LINE_AA)
    
    # PREDICT IMAGE
    predictions = model.predict(img)
    classIndex = np.argmax(predictions)
    probabilityValue = np.max(predictions)
    
    if probabilityValue > threshold:
        cv2.putText(imgOrignal, str(classIndex) + " " + str(getClassName(classIndex)),
                    (120, 35), font, 0.75, (0, 0, 255), 2, cv2.LINE_AA)
        cv2.putText(imgOrignal, str(round(probabilityValue * 100, 2)) + "%", (180, 75),
                    font, 0.75, (0, 0, 255), 2, cv2.LINE_AA)
        
    cv2.imshow("Result", imgOrignal)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 274ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━

KeyboardInterrupt: 

: 